In [1]:
import numpy as np
import pandas as pd

import pandapower as pp
import pandapower.control as control
import pandapower.networks as nw
import pandapower.timeseries as timeseries
from pandapower.timeseries import OutputWriter, DFData
from pandapower.control import ConstControl

# Load and prepare the Pandapower network
net = nw.create_cigre_network_lv()

# Set switches and other parameters as needed
net.switch.loc[1, "closed"] = False
net.switch.loc[2, "closed"] = False
for load_idx in net.load.index:
    net.load.at[load_idx, 'controllable'] = False
for line_idx in net.line.index:
    net.line.at[line_idx, 'max_loading_percent'] = 100


In [ ]:
# Load and preprocess the PV generation profile CSV file
df_pv = pd.read_csv("pv_generation_profile.csv")
df_pv['time'] = pd.to_datetime(df_pv['time'], format='%H:%M:%S').dt.time
df_pv['time_step'] = range(len(df_pv))  # Create a numerical index
df_pv.set_index('time_step', inplace=True)
df_pv['pvgen'] = df_pv['pvgen'] * 250 / 1000
ds_pv = DFData(df_pv)

In [ ]:
# Load and preprocess the load profile CSV file
df = pd.read_csv("load_profile_1111.csv")
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time
df['time_step'] = range(len(df))  # Create a numerical index
df.set_index('time_step', inplace=True)
df['mult'] = df['mult'] * 15 / 1000
ds = DFData(df)

In [ ]:
### SCRIPTS ###
import param as pm
import data as dt
from data import bus,sets,load,bess,gen,inverter,branch,cost,oltc,pv,ts, COP, hd

In [ ]:
import gurobipy as gp
import numpy as np

def create_opf_model():
    m = gp.Model('OPF')
    
    # Define your variables
    pR = m.addVars(len(net.bus), name='pR')
    qR = m.addVars(len(net.bus), name='qR')
    vRe = m.addVars(len(net.bus), name='vRe')
    vIm = m.addVars(len(net.bus), name='vIm')
    # ... other variables
    
    # Define constraints
    # Example: Voltage constraints
    m.addConstrs((vRe[i] <= 1.1 for i in range(len(net.bus))), "VoltageUpperBound")
    m.addConstrs((vRe[i] >= 0.9 for i in range(len(net.bus))), "VoltageLowerBound")
    # ... other constraints
    
    # Define objective function
    m.setObjective(gp.quicksum(pR[i] for i in range(len(net.bus))), gp.GRB.MINIMIZE)
    
    return m

opf_model = create_opf_model()

In [ ]:
def run_load_flow(net):
    pp.runpp(net)
    return net

def update_opf_model(opf_model, net):
    # Update variables and constraints in the OPF model based on new load flow results
    for i, bus in net.bus.iterrows():
        opf_model.getVarByName(f"vRe[{i}]").lb = net.res_bus.vm_pu.at[i] 
        opf_model.getVarByName(f"vRe[{i}]").ub = net.res_bus.vm_pu.at[i] 
        # ... other updates

In [ ]:
def iterative_opf(net, opf_model, max_iterations=10, tolerance=1e-3):
    for iteration in range(max_iterations):
        # Run load flow
        net = run_load_flow(net)
        
        # Update OPF model with new states
        update_opf_model(opf_model, net)
        
        # Optimize
        opf_model.optimize()
        
        # Check for convergence (Example: based on the objective function or variables change)
        if opf_model.status == gp.GRB.OPTIMAL:
            if iteration > 0 and abs(previous_obj_value - opf_model.objVal) < tolerance:
                print("Converged!")
                break
            previous_obj_value = opf_model.objVal
        else:
            print("Optimization failed")
            break

    return net, opf_model

# Run the iterative optimization process
final_net, final_opf_model = iterative_opf(net, opf_model)